#BERTopic on Quran Urdu Translations
In this notebook I have implemented Topic Modeling on Urdu translations based topic modelling technique BERTopic.

Shaista Zulfiqar

## Mounting Google Drive
If the dataset is on Google Drive then you have to mount over google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




#Installing required dependencies
**One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.**

In [ ]:
!pip install bertopic
!pip install -U sentence-transformers
!pip install urduhack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 M

In [ ]:
!pip install --upgrade keras tensorflow-addons


# Importing required dependencies
We will import numpy, pandas and re, bertopic, gensim library for now. other libraries will be imported in the notebook later.

Pandas will be used to create a Dataframe and handle the csv file. Numpy will be used for the faster computation of arrays to save time. re library will be used for the cleaning of data. gensim library is used to get coherence score and train LDA. bertopic is used to train bertopic on our Quran-UTM dataset with using pretrained language models Multilingial MiniLM

In [ ]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from urduhack.normalization import remove_diacritics
from gensim.models import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
#optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

##DataFrame




In [ ]:

junapd = pd.read_table("please provide your (.txt) data path", header=None, encoding='utf-8')

print(junapd.head(5))

stopspd=pd.read_csv('/content/drive/MyDrive/stopwords.txt',names=['List']) # your path

stopspd

                                                   0
0       1|1|عظیم اور دائمی رحمتوں والے خدا کے نام سے
1  1|2|ساری تعریف اللہ کے لئے ہے جو عالمین کا پال...
2                1|3|وہ عظیم اوردائمی رحمتوں والا ہے
3                   1|4|روزِقیامت کا مالک و مختار ہے
4  1|5|پروردگار! ہم تیری ہی عبادت کرتے ہیں ا ور ت...


,List
0,کی
1,ہیں
2,ہے
3,رہا
4,رہی
...,...
396,گئی
397,ہونے
398,وجہ
399,ہوگیا


## Cleaning of Data
After collecting the eight Quran Urdu translations, we observed these translation had some irrelevant information, such as metadata, punctuation, and diacritics. So we cleaned the translations for topic modeling

Stopwords are common words that are often filtered out during text processing in natural language processing (NLP) tasks. These words are considered to have little or no value in conveying the actual meaning of the text. We take list of 401 stopwords for topic modelling. Stopwords are removed in post preprocessing phase

In [ ]:
import re
#Remove Urdu Punctuation
def remove_urdu_punctuation(text):
    pattern = r'[؛؟،٫٬‘’“”«»!"٪&\'\*\+,-./:;<=>؟@^_`()[]{|}~]'

    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text


In [ ]:
# Remove trailing metadata lines
index_to_drop = junapd[junapd[0].str.startswith("#")].index
junapd.drop(index_to_drop, inplace=True)

# Remove verse reference from every line
junapd[0] = junapd[0].str.replace(r'\d+\|\d+\|', '', regex=True)

# Remove diacritics
junapd[0] = junapd[0].apply(remove_diacritics)

# Remove punctuation
junapd[0] = junapd[0].apply(remove_urdu_punctuation)

# Reset index
junapd.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(junapd.head())



Cleaned DataFrame:
                                                   0
0           عظیم اور دائمی رحمتوں والے خدا کے نام سے
1  ساری تعریف اللہ کے لئے ہے جو عالمین کا پالنے و...
2                    وہ عظیم اوردائمی رحمتوں والا ہے
3                        روزقیامت کا مالک و مختار ہے
4  پروردگار! ہم تیری ہی عبادت کرتے ہیں ا ور تجھی ...


In [ ]:
def remove_nonbreaking_space(text):
    return re.sub(r'\xa0', ' ', text)

junapd[0] = junapd[0].apply(remove_nonbreaking_space)

In [ ]:
# Save the cleaned DataFrame to a text file
file_path = 'cleaned_Jawadi_text.txt'
junapd[0].to_csv(file_path, sep='\n', index=False, header=False, encoding='utf-8')

print(f'Text has been saved to {file_path}')

Text has been saved to cleaned_Jawadi_text.txt


In [ ]:
# Convert the DataFrame to a list of strings
data = junapd[0].tolist()

In [ ]:
print(len(data))

6236


In [ ]:
print(data[56:90])

['ہمارا یہ احسان بھی یادکرو کہ ہم نے دریا کو شگافتہ کرکے تمہیں بچالیا اور فرعون والوں کو تمہاری نگاہوں کے سامنے ڈبو دیا', 'اور ہم نے موسی علیھ السّلام سے چالیس راتوں کا وعدہ لیا تو تم نے ان کے بعد گو سالہ تیار کرلیا کہ تم بڑے ظالم ہو', 'پھر ہم نے تمہیں معاف کردیا کہ شاید شکر گزار بن جاؤ', 'اور ہم نے موسی علیھ السّلام کو کتاب اور حق و باطل کو جدا کرنے والا قانون دیا کہ شاید تم ہدایت یافتہ بن جاؤ', 'اور وہ وقت بھی یاد کرو جب موسی علیھ السّلام نے اپنی قوم سے کہا کہ تم نے گو سالہ بنا کر اپنے اوپر ظلم کیا ہے. اب تم خالق کی بارگاہ میں توبہ کرو اور اپنے نفسوں کو قتل کر ڈالو کہ یہی تمہارے حق میں خیر ہے. پھر خدا نے تمہاری توبہ قبول کرلی کہ وہ بڑا توبہ قبول کرنے والا مہربان ہے', 'اور وہ وقت بھی یاد کرو جب تم نے موسی علیھ السّلام سے کہا کہ ہم اس وقت تک ایمان نہ لائیں گے جب تک خدا کو اعلانیہ نہ دیکھ لیں جس کے بعد بجلی نے تم کو لے ڈالا اور تم دیکھتے ہی رہ گئے', 'پھر ہم نے تمہیں موت کے بعد زندہ کردیا کہ شاید اب شکر گزار بن جاؤ', 'اور ہم نے تمہارے سروں پر ابر کا سایہ کیا. تم پر من و سلوی نازل کیا کہ 

# BERTopic Training
The default  bertopic embedding model is paraphrase-multilingual-MiniLM-L12-v2 when selecting language="multilingual". We take MiniLM Model from [sentence-tranformer](https://www.sbert.net/docs/pretrained_models.html) and create custom document embedding and passed it to the bertopic model for training.

In [ ]:
#create custom embedding
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(data, show_progress_bar=True)
print(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/195 [00:00<?, ?it/s]

[[-0.03584485  0.428462   -0.04711394 ... -0.11707891  0.23166981
   0.09169383]
 [-0.0494466   0.23540665 -0.0363511  ... -0.09146018  0.09004427
   0.0598713 ]
 [-0.07855674  0.39043304 -0.04762787 ... -0.24353735  0.2970589
   0.06023435]
 ...
 [ 0.01638935  0.3803766  -0.16181065 ... -0.12230271  0.25334242
   0.07125952]
 [-0.03681972 -0.08630634 -0.10165572 ...  0.01698185  0.18368855
  -0.06030149]
 [-0.10259257  0.14364305 -0.03021561 ... -0.07043492  0.20209667
  -0.10529657]]


In [ ]:
stop_words_list = stopspd['List'].tolist()

In [ ]:
#pass vectorizer_model to bertopic for stopwords removal
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words= stop_words_list)


In [ ]:
#UMAP for dimention reduction
from umap import UMAP
dim_model = UMAP(n_components=4, random_state=42)

In [ ]:
# # #KMeans used for clustering
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=5, random_state=42)

In [ ]:
np.random.seed(42)

In [ ]:
topic_model = BERTopic(language="urdu", low_memory=True ,calculate_probabilities=True,  top_n_words=10,hdbscan_model=cluster_model, umap_model=dim_model, verbose=True, vectorizer_model=vectorizer_model)

In [ ]:
#Fit documents in bertopic
topics, probs = topic_model.fit_transform(data,embeddings)

2024-06-09 13:51:33,709 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-09 13:52:20,782 - BERTopic - Dimensionality - Completed ✓
2024-06-09 13:52:20,785 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-09 13:52:20,914 - BERTopic - Cluster - Completed ✓
2024-06-09 13:52:20,929 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-09 13:52:21,129 - BERTopic - Representation - Completed ✓


In [ ]:
print(probs)

None


In [ ]:
#topics that assign to each document
print(topics)

[1, 0, 1, 1, 1, 0, 0, 0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 3, 2, 0, 1, 1, 1, 2, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 1, 1, 1, 3, 1, 2, 0, 1, 0, 1, 3, 1, 0, 0, 0, 1, 1, 2, 2, 2, 1, 3, 1, 2, 0, 1, 3, 1, 2, 0, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 0, 2, 2, 2, 3, 2, 1, 3, 1, 3, 1, 2, 1, 1, 1, 2, 1, 2, 3, 0, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 3, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 1, 2, 2, 0, 2, 1, 1, 0, 2, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 2, 1, 0, 2, 1, 2, 0, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 3, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 0, 0, 2, 1, 2, 2, 2, 2, 0, 2, 3, 1, 1, 1, 1, 1, 2, 0, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 0, 1, 1, 1, 2, 2, 2, 2, 1, 2, 3, 1, 2, 0, 

In [ ]:
topic_model.get_topic_freq()

,Topic,Count
1,0,2392
0,1,2294
3,2,1166
2,3,351
4,4,33


In [ ]:
document_topics = topic_model.get_topics()

In [ ]:
#topics with score
print(document_topics)

{0: [('دن', 0.044067246844784254), ('کوئی', 0.0348236451799697), ('عذاب', 0.0318111361184155), ('لوگوں', 0.03164520577438548), ('لوگ', 0.031301236146403025), ('کردیا', 0.02633682903983926), ('تمہارے', 0.0255380315703856), ('والوں', 0.024718753768716028), ('پیدا', 0.024630771340664194), ('زمین', 0.02460728579855068)], 1: [('خدا', 0.08189246228077289), ('پروردگار', 0.05065458332336725), ('لوگوں', 0.036863191635769746), ('والا', 0.03637567221272458), ('ایمان', 0.03456709346308556), ('لوگ', 0.03429134105653), ('کوئی', 0.0338001003464663), ('تمہارے', 0.030056487201305312), ('عذاب', 0.0251064848905677), ('زمین', 0.024017854553495473)], 2: [('اللہ', 0.12079362612486493), ('خدا', 0.05030797769536785), ('والا', 0.03958230191386277), ('ایمان', 0.038288055459525516), ('تمہارے', 0.036664616151912875), ('پیغمبر', 0.035392235787213074), ('علیھ', 0.03427365125954663), ('الس', 0.034224875623853575), ('لوگ', 0.03369213232572012), ('کوئی', 0.03262265109475541)], 3: [('موسی', 0.1566362493508074), ('فرعون

In [ ]:
#shows which documents assign to specific topic
topic_model.get_representative_docs(topic=None)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2392,0_دن_کوئی_عذاب_لوگوں,"[دن, کوئی, عذاب, لوگوں, لوگ, کردیا, تمہارے, وا...",[اور سلام ہے مجھ پر اس دن جس دن میں پیدا ہوا ا...
1,1,2294,1_خدا_پروردگار_لوگوں_والا,"[خدا, پروردگار, لوگوں, والا, ایمان, لوگ, کوئی,...",[جب ان سے کہا جاتا ہے کہ خدا کے نازل کئے ہوئے ...
2,2,1166,2_اللہ_خدا_والا_ایمان,"[اللہ, خدا, والا, ایمان, تمہارے, پیغمبر, علیھ,...",[یقینا ہم نے نوح علیھ السّلامکو ان کی قوم کی ط...
3,3,351,3_موسی_فرعون_الس_علیھ,"[موسی, فرعون, الس, علیھ, قوم, کتاب, لام, نوح, ...",[پھر ان سب کے بعد ہم نے موسی علیھ السّلامکو اپ...
4,4,33,4_نعمت_رب_انکار_جھٹلاؤ,"[نعمت, رب, انکار, جھٹلاؤ, پروردگار, دونوں, کن,...","[پھر تم اپنے رب کی کس کس نعمت کا انکار کرو گے,..."


In [ ]:
topic_model.get_document_info(data)

In [ ]:
topic_distr, _ = topic_model.approximate_distribution(data, window=3, min_similarity=0.01)

100%|██████████| 7/7 [00:02<00:00,  3.37it/s]


In [ ]:
print(topic_distr)

[[0.03822089 0.53692015 0.2940873  0.100271   0.03050066]
 [0.07245244 0.17079692 0.64026699 0.11648366 0.        ]
 [0.18672733 0.40346785 0.36761135 0.04219347 0.        ]
 ...
 [0.19082783 0.42924175 0.23334933 0.12891128 0.0176698 ]
 [0.31078198 0.32660919 0.20960492 0.15300391 0.        ]
 [0.29557473 0.29746631 0.23529667 0.17166228 0.        ]]


In [ ]:
topic_model.visualize_distribution(topic_distr[0], width=600,height=600, title="Topic Probability Distributio")

# Evaluation
we used three evaluation metrics to compare the results.

1. The coherence score is used to capture the degree of similarity between the words within each topic, with higher scores indicating more coherent topics. We used two coherence metrics NPMI and Cv Score.
2. IRBO measures are used to assess how different and distinct the topics are in a topic model.


### Coherence Score
To evaluate the model topics coherence we use [Gensim](https://radimrehurek.com/gensim/models/coherencemodel.html) library

In [ ]:
texts = [[word for word in str(document).split() if word not in stop_words_list] for document in data] #if word not in stop_words_list
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
topics_bert=[]
for i in topic_model.get_topics():
  row=[]
  topic= topic_model.get_topic(i)
  for word in topic:
     row.append(word[0])
  topics_bert.append(row)

In [ ]:
print(topics_bert)

[['دن', 'کوئی', 'عذاب', 'لوگوں', 'لوگ', 'کردیا', 'تمہارے', 'والوں', 'پیدا', 'زمین'], ['خدا', 'پروردگار', 'لوگوں', 'والا', 'ایمان', 'لوگ', 'کوئی', 'تمہارے', 'عذاب', 'زمین'], ['اللہ', 'خدا', 'والا', 'ایمان', 'تمہارے', 'پیغمبر', 'علیھ', 'الس', 'لوگ', 'کوئی'], ['موسی', 'فرعون', 'الس', 'علیھ', 'قوم', 'کتاب', 'لام', 'نوح', 'اسرائیل', 'بنی'], ['نعمت', 'رب', 'انکار', 'جھٹلاؤ', 'پروردگار', 'دونوں', 'کن', 'دعوت', 'شپھر', 'دکھلاتا']]


In [ ]:
# compute Coherence Score CV

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_v')
coherence = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence)


CV Score:  0.49


In [ ]:
# compute Coherence Score NPMI

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_npmi')
coherence = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence)


NPMI Score:  -0.01


**Diversity Score**

upload rbo.py file before importing

In [ ]:
import itertools
from rbo import rbo
import numpy as np

class InvertedRBO:
    def __init__(self):
        pass

    def irbo(self, topics, topk=10, weight=0.9):
        """
        Calculate inverted Rank Biased Overlap (RBO) as a measure of topic diversity from a list of lists of words.

        :param topics: A list of lists of words representing different topics.
        :param topk: The number of top words on which RBO will be computed.
        :param weight: Weight of each agreement at depth d: p**(d-1). When set to 1.0, there is no weight,
                       and the RBO returns to average overlap.
        :return: The inverted RBO topic diversity score.
        """
        if topk <= 0:
            raise ValueError("topk must be a positive integer.")

        num_topics = len(topics)
        if num_topics == 0:
            raise ValueError("topics list cannot be empty.")

        if topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')

        collect = []
        for list1, list2 in itertools.combinations(topics, 2):
            rbo_val = rbo(list1[:topk], list2[:topk], p=weight)[2]
            collect.append(rbo_val)

        Irbo_score = 1 - np.mean(collect)
        return Irbo_score

In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= round(inverted_rbo_calculator.irbo(topics_bert, topk=10, weight=0.9),2)
print("Inverted RBO Score:", IRBO)

Inverted RBO Score: 0.88


# Visualize Topics

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(n_words=10,width=220, height=270, title="Topic Word Scores")

# Model serialization

In [ ]:
# Save model
topic_model.save("my_model")

2024-06-09 10:52:31,148 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
loaded_topic_model = topic_model.load("my_model")
loaded_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2476,0_بیشک_دن_تمہیں_کوئی,"[بیشک, دن, تمہیں, کوئی, تمہارے, پاس, زمین, عذا...",[اور وہی ہے جس نے تمہیں زندہ کیا اور پھر تمہیں...
1,1,1892,1_اللہ_بیشک_تمہیں_تمہارے,"[اللہ, بیشک, تمہیں, تمہارے, ایمان, والا, کوئی,...","[تم فرماؤ وہ اللہ ہے وہ ایک ہے, اے رسول پہنچا ..."
2,2,1593,2_رب_بیشک_اے_ایمان,"[رب, بیشک, اے, ایمان, والا, تمہارے, پاس, میرے,...",[عرض کی، اے میرے رب! میں نے اپنی جان پر زیادتی...
3,3,243,3_موسی_فرعون_قوم_بیشک,"[موسی, فرعون, قوم, بیشک, نوح, اے, اسرائیل, بنی...","[اور تو نے اپنی قوم سے کیوں جلدی کی اے موسی, ا..."
4,4,32,4_جھٹلاؤ_کونسی_نعمت_رب,"[جھٹلاؤ, کونسی, نعمت, رب, دونوں, سی, کون, انس,...","[تو اپنے رب کی کونسی نعمت جھٹلاؤ گے،, تو اپنے ..."


# LDA

We use the [ parallelized Latent Dirichlet Allocation (LDA)](https://radimrehurek.com/gensim/models/ldamulticore.html) from Gensim.

Note: for LDA you have to define topics number in advance.

In [ ]:
n_topics=5
lda = LdaMulticore(corpus, id2word=id2word, random_state=42, num_topics=5)
topics = lda.show_topics(num_topics=5, formatted=False)

#Extract the words from the topics
topics_list = []
for _, topic_words in topics:
    words = [word for word, _ in topic_words]
    topics_list.append(words)

print(topics_list)

In [ ]:
#CV Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_lda = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_lda)


CV Score:  0.48


In [ ]:
#NPMI Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_lda = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_lda)


NPMI Score:  -0.02


In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_LDA = round(inverted_rbo_calculator.irbo(topics_list, topk=10, weight=0.9),2)
print('\nIRBO Score: ',IRBO_LDA)


IRBO Score:  0.65


#NMF
We use Gensim library for implementation of NMF

Note: for NMF you have to define topics number in advance.

In [ ]:
#Using Gensim
from gensim import corpora, models


# Train NMF model
num_topics = 5  # Define the number of topics
nmf_model = models.Nmf(corpus, num_topics=num_topics, id2word=id2word, random_state=42)

# Extract topics
topics = []
for topic_id in range(num_topics):
    topic_words = nmf_model.show_topic(topic_id, topn=10)
    topic_words = [word for word, _ in topic_words]
    topics.append(topic_words)

# Print topics in the desired format
print(topics)

In [ ]:
#Calculate CV Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_nmf = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_nmf)

In [ ]:
#Calculate NPMI Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_nmf = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_nmf)

In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_NMF = round(inverted_rbo_calculator.irbo(topics, topk=10, weight=0.9),2)c
print('\nIRBO Score: ',IRBO_NMF)